<a href="https://colab.research.google.com/github/JorgeSedek/tp3_orga_datos/blob/main/tp3_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from xgboost import XGBClassifier 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df_train = pd.read_parquet("/content/drive/MyDrive/orgaDatos/TP3 dataset music/train.parquet")

In [ ]:
df_test = pd.read_parquet("/content/drive/MyDrive/orgaDatos/TP3 dataset music/test.parquet")

In [ ]:
df_train.count()

track_name          31383
lyric               31380
genre               31383
language            24021
popularity          31383
artist              31383
a_genres            31383
a_songs             31383
a_popularity        31383
did                  7004
s-label              7004
acousticness        31383
danceability        31383
duration_ms         31383
energy              31383
instrumentalness    31383
key                 31383
liveness            31383
loudness            31383
mode                31383
speechiness         31383
tempo               31383
time_signature      31383
valence             31383
dtype: int64

In [ ]:
def fix_generos(x):
    if x in ["Children’s Music"]:
        return "Children's Music"
    return x
 

df_train.genre = df_train.genre.map(lambda x:fix_generos(x))
df_test.genre = df_test.genre.map(lambda x:fix_generos(x))

In [ ]:
df_train.drop(['track_name','a_genres','did','time_signature'], axis=1, inplace= True)
df_test.drop(['track_name','a_genres','did','time_signature'], axis=1, inplace= True)

In [ ]:
idioma_mas_popular = df_train['language'].value_counts().reset_index()['index'][0]

In [ ]:
df_train['language'] = df_train['language'].fillna(idioma_mas_popular)
df_test['language'] = df_test['language'].fillna(idioma_mas_popular)

In [ ]:
df_train['s-label'] = df_train['s-label'].fillna(df_train['s-label'].mean())
df_test['s-label'] = df_test['s-label'].fillna(df_test['s-label'].mean())

In [ ]:
df_train.count()

lyric               31380
genre               31383
language            31383
popularity          31383
artist              31383
a_songs             31383
a_popularity        31383
s-label             31383
acousticness        31383
danceability        31383
duration_ms         31383
energy              31383
instrumentalness    31383
key                 31383
liveness            31383
loudness            31383
mode                31383
speechiness         31383
tempo               31383
valence             31383
dtype: int64

# Separación de Test y Valid

Para separar el test y valid se utiliza la feature artist:

*   Se mezcla al azar la lista de artistas sin repetir
*   Se toma el 80% de esos artistas
*   se divide el set de entrenamiento:

    1.  las filas que tienen uno de los artistas dentro de la muestra del 80% pasan a ser el nuevo set de entrenamiento.
    2.  el resto pasa a ser el set de validación.

In [ ]:
artistas_shuffled = shuffle(df_train['artist'].unique(), random_state = 8)
artistas_80 = artistas_shuffled[:8*(len(artistas_shuffled)//10)]
len(artistas_80) / len(artistas_shuffled)

0.7985074626865671

In [ ]:
df_train_final = df_train[df_train['artist'].map(lambda x: x in artistas_80)]
df_valid_final = df_train[df_train['artist'].map(lambda x: not (x in artistas_80))]

df_train_final.drop(['artist'], axis=1, inplace= True)
df_valid_final.drop(['artist'], axis=1, inplace= True)
df_test.drop(['artist'], axis=1, inplace= True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
y_train = df_train_final.genre
y_test = df_test.genre
x_train = df_train_final.drop("genre", axis = 1)
x_test = df_test.drop("genre", axis = 1)

In [ ]:
y_valid = df_valid_final.genre
x_valid = df_valid_final.drop("genre", axis = 1)

In [ ]:
sets_trabajo = [x_train, x_valid, x_test]

# Encoding para lyric

In [ ]:
from nltk.corpus import stopwords

import nltk

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from nltk import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer

In [ ]:
stopwordsEN = stopwords.words('english')

In [ ]:
lemma = WordNetLemmatizer()

def lematizar(texto):
  texto_lematizado= ""
  for palabra in word_tokenize(texto):
      texto_lematizado = texto_lematizado + " " + lemma.lemmatize(palabra)
  return texto_lematizado

def filtrar(texto):
  caracteres_eliminar = "./,``,’?!#$%&/()]_-[{}0123456789²'"
  
  for caracter in caracteres_eliminar:
    texto = texto.replace(caracter,"")
    
  return texto

In [ ]:
vectorizer = CountVectorizer(lowercase=True, stop_words=stopwordsEN)
textos = ""
for line in x_test.lyric:
    textos = textos + filtrar(lematizar(str(line)).lower().strip(',.!"#$%/(=?)¡[]{}"'))

In [ ]:
matrix = vectorizer.fit_transform([textos])
matrix

<1x13755 sparse matrix of type '<class 'numpy.int64'>'
	with 13755 stored elements in Compressed Sparse Row format>

In [ ]:
counts = pd.DataFrame(matrix.toarray(),
                      columns=vectorizer.get_feature_names())
counts

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aaaaaaaaaaaaaaaaooooh,aaaaaaaaaaah,aah,aahh,aba,abandon,abandoned,abba,aberration,abide,...,zonia,zoo,zooky,zoom,zoot,zu,zulu,zutto,às,ëround
0,1,3,8,10,1,13,6,2,1,6,...,1,4,1,3,2,6,3,8,2,2


In [ ]:
palabras_freq = counts.T.sort_values(by=0, ascending=False).head(20)
palabras_freq2 = palabras_freq[0].reset_index().rename(columns={"index": "palabras"})
palabras_frecuentes = palabras_freq2['palabras'].tolist()
palabras_frecuentes

['nt',
 'like',
 'know',
 'love',
 'got',
 'na',
 'get',
 'go',
 'yeah',
 'oh',
 'time',
 'never',
 'want',
 'wa',
 'baby',
 'come',
 'let',
 'ca',
 'one',
 'cause']

In [ ]:
for set_trabajo in sets_trabajo:
  for palabra in palabras_frecuentes:
    set_trabajo[str(palabra)] = set_trabajo['lyric'].map(lambda x: 1 if (str(palabra) in str(x)) else 0)

In [ ]:
x_train = x_train.reset_index().drop(['lyric'],axis=1)
x_valid = x_valid.reset_index().drop(['lyric'],axis=1)
x_test = x_test.reset_index().drop(['lyric'],axis=1)

# One-hot encoding de variables categoricas

In [ ]:
features_ohe = ['language','mode']

In [ ]:
def cant_columns_num(x_set):
  cant_columns_num = 0
  for columna in x_set.columns:
    if isinstance(columna, int):
      cant_columns_num += 1
  return cant_columns_num

def renombrar(nombre_feature, x_set):
  for i in range(cant_columns_num(x_set)):
    x_set = x_set.rename(columns = {i: (nombre_feature + str(i))})
  return x_set

In [ ]:
for nombre_feature in features_ohe:
  ohe = OneHotEncoder(drop='first', handle_unknown='ignore')
  encoded = ohe.fit_transform(x_train[[nombre_feature]]).todense().astype(int)
  encoded_valid = ohe.transform(x_valid[[nombre_feature]]).todense().astype(int)
  encoded_test = ohe.transform(x_test[[nombre_feature]]).todense().astype(int)

  x_train = x_train.reset_index().drop(['index', nombre_feature],axis=1).join(pd.DataFrame(encoded))
  x_valid = x_valid.reset_index().drop(['index', nombre_feature],axis=1).join(pd.DataFrame(encoded_valid))
  x_test = x_test.reset_index().drop(['index', nombre_feature],axis=1).join(pd.DataFrame(encoded_test))
  
  x_train = renombrar(nombre_feature,x_train)
  x_valid = renombrar(nombre_feature,x_valid)
  x_test = renombrar(nombre_feature,x_test)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:174: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  UserWarning,


In [ ]:
x_train.columns

Index(['level_0', 'popularity', 'a_songs', 'a_popularity', 's-label',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'speechiness',
       'tempo', 'valence', 'nt', 'like', 'know', 'love', 'got', 'na', 'get',
       'go', 'yeah', 'oh', 'time', 'never', 'want', 'wa', 'baby', 'come',
       'let', 'ca', 'one', 'cause', 'language0', 'language1', 'language2',
       'language3', 'language4', 'language5', 'language6', 'language7',
       'language8', 'language9', 'mode0'],
      dtype='object')

In [ ]:
ohe_genre= OneHotEncoder(drop='first', handle_unknown='ignore')
encoded_genre = ohe_genre.fit_transform(pd.DataFrame(y_train)).todense().astype(int)
encoded_genre_valid = ohe_genre.transform(pd.DataFrame(y_valid)).todense().astype(int)
encoded_genre_test = ohe_genre.transform(pd.DataFrame(y_test)).todense().astype(int)

y_train2 = y_train.reset_index().drop(['index', 'genre'],axis=1).join(pd.DataFrame(encoded_genre))
y_valid2 = y_valid.reset_index().drop(['index', 'genre'],axis=1).join(pd.DataFrame(encoded_genre_valid))
y_test2 = y_test.reset_index().drop(['index', 'genre'],axis=1).join(pd.DataFrame(encoded_genre_test))

# Mean Encoding

In [ ]:
key_mode = x_train.loc[:,["mode0", 'key']].groupby(['key']).mean().sort_values(by="mode0",ascending=False).reset_index()
key_mode

,key,mode0
0,B,0.512075
1,A#,0.489946
2,E,0.487310
3,F#,0.483851
4,F,0.419652
5,D#,0.331558
6,A,0.320377
7,C#,0.300000
8,D,0.202594
9,G#,0.198119


In [ ]:
set_features = [x_train, x_valid, x_test]

In [ ]:
def mean_encoding(key):
  return key_mode[key_mode['key'].map(lambda x: x == key)]['mode0'].values[0]

In [ ]:
for features in set_features:
  features['key'] = features['key'].map(lambda x: mean_encoding(x))

In [ ]:
x_train['key']

0        0.512075
1        0.512075
2        0.512075
3        0.512075
4        0.512075
           ...   
25237    0.487310
25238    0.487310
25239    0.483851
25240    0.154978
25241    0.419652
Name: key, Length: 25242, dtype: float64

In [ ]:
x_train = x_train.reset_index().drop(['index', 'level_0'],axis=1)
x_valid = x_valid.reset_index().drop(['index', 'level_0'],axis=1)
x_test = x_test.reset_index().drop(['index', 'level_0'],axis=1)

# Random Forest

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

In [ ]:
model = RandomForestClassifier(criterion = 'entropy', random_state= 8)

In [ ]:
# define search space
space = {}
space['criterion'] = ['gini', 'entropy', 'log_loss']

In [ ]:
search = RandomizedSearchCV(model, space, n_iter=2, scoring='accuracy', n_jobs= 1, cv=2, random_state= 8, verbose =10)

In [ ]:
# execute search
result = search.fit(x_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV 1/2; 1/2] START criterion=log_loss..........................................
[CV 1/2; 1/2] END .............criterion=log_loss;, score=nan total time=   0.1s
[CV 2/2; 1/2] START criterion=log_loss..........................................
[CV 2/2; 1/2] END .............criterion=log_loss;, score=nan total time=   0.1s
[CV 1/2; 2/2] START criterion=entropy...........................................
[CV 1/2; 2/2] END ............criterion=entropy;, score=0.424 total time=  10.3s
[CV 2/2; 2/2] START criterion=entropy...........................................
[CV 2/2; 2/2] END ............criterion=entropy;, score=0.389 total time=  10.3s


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
2 fits failed out of a total of 4.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 1085, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/paralle

Best Score: 0.4063069487362333
Best Hyperparameters: {'criterion': 'entropy'}


In [ ]:
model.fit(x_train, y_train)

RandomForestClassifier(criterion='entropy', random_state=8)

In [ ]:
top_k_accuracy_score(y_valid, model.predict_proba(x_valid), k=2, labels = model.classes_)

0.5308581664224068

In [ ]:
model.feature_importances_

array([1.86037495e-01, 6.80352618e-02, 7.40295895e-02, 1.36802629e-02,
       6.03422593e-02, 5.39842069e-02, 4.28352149e-02, 5.19908764e-02,
       4.44782663e-02, 2.11764869e-02, 2.85431703e-02, 5.08249493e-02,
       5.81231891e-02, 3.03471816e-02, 3.77856738e-02, 6.22398522e-03,
       1.06779883e-02, 8.98386698e-03, 9.24512082e-03, 1.05365679e-02,
       8.96678257e-03, 1.07022433e-02, 6.93325339e-03, 6.79621237e-03,
       7.25476244e-03, 8.37290876e-03, 7.09884032e-03, 7.07529425e-03,
       6.60472174e-03, 8.95154383e-03, 6.94104573e-03, 7.27118413e-03,
       5.38941764e-03, 7.58637170e-03, 5.13756696e-03, 1.62456708e-06,
       6.83112171e-03, 5.28797481e-03, 3.79901725e-05, 5.92139638e-07,
       1.11295568e-05, 1.49562444e-04, 2.14162517e-04, 5.38023605e-05,
       9.23054469e-08, 8.44818513e-03])

In [ ]:
top_k_accuracy_score(y_test, model.predict_proba(x_test), k=2, labels = model.classes_)

0.495260663507109

In [ ]:
prediciones = model.predict_proba(x_train)

In [ ]:
x_train2 = x_train.copy()

In [ ]:
i = 0
for genero in model.classes_:
  x_train2[genero] = prediciones[:,i]
  i+= 1

In [ ]:
x_train2

,popularity,a_songs,a_popularity,s-label,acousticness,danceability,duration_ms,energy,instrumentalness,key,...,Pop,R&B,Rap,Reggae,Reggaeton,Rock,Ska,Soul,Soundtrack,World
0,79,276.0,205.5,0.490309,0.2720,0.508,261640,0.720,0.000000,0.512075,...,0.194500,0.185500,0.0,0.00,0.0,0.00,0.00,0.000,0.00,0.0000
1,80,276.0,205.5,0.490309,0.2720,0.508,261640,0.720,0.000000,0.512075,...,0.450000,0.430000,0.0,0.00,0.0,0.00,0.00,0.000,0.00,0.0000
2,80,276.0,205.5,0.490309,0.2720,0.508,261640,0.720,0.000000,0.512075,...,0.450000,0.430000,0.0,0.00,0.0,0.00,0.00,0.000,0.00,0.0000
3,71,276.0,205.5,0.490309,0.0296,0.412,319467,0.441,0.072600,0.512075,...,0.343563,0.329468,0.0,0.00,0.0,0.01,0.00,0.000,0.00,0.0000
4,71,276.0,205.5,0.490309,0.0296,0.412,319467,0.441,0.072600,0.512075,...,0.343563,0.329468,0.0,0.00,0.0,0.01,0.00,0.000,0.00,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25237,19,89.0,1.0,0.748000,0.6120,0.616,190733,0.822,0.000003,0.487310,...,0.003333,0.010667,0.0,0.05,0.0,0.01,0.03,0.026,0.00,0.7200
25238,34,89.0,1.0,0.868000,0.8890,0.457,230200,0.369,0.011100,0.487310,...,0.000000,0.000000,0.0,0.01,0.0,0.00,0.00,0.010,0.00,0.8475
25239,18,89.0,1.0,0.793000,0.7630,0.717,275640,0.566,0.812000,0.483851,...,0.000000,0.000000,0.0,0.01,0.0,0.00,0.00,0.010,0.01,0.8200
25240,28,89.0,1.0,0.738000,0.8040,0.633,204373,0.553,0.000866,0.154978,...,0.000000,0.000000,0.0,0.03,0.0,0.00,0.01,0.020,0.00,0.7500


In [ ]:
x_train2.to_csv('predicciones_random_forest.csv', index=False)